In [8]:

function spectral_density(J,Nb,Γ,D,ϵi)
    ###Need to sort V_k the same way as the energies
    V_k = sqrt(Γ/(2*D))*ones(Nb,1) 
    if Nb>1
        if J== "boxcar"
            V_k = sqrt(Γ/(2*D))*ones(Nb,1)                       # Couplings
            ϵb = collect(range(-1, 1, length = Nb))              # energy of bath modes
        end
        ind = sortperm(abs.(ϵb.-ϵi[Ns]))                               # Broadcast abs() over vector using "." notation.
        V_k, ϵb = V_k[ind], ϵb[ind];  
    else
        ϵb = 0;
    end
    return V_k, ϵb
end

J_box(w,V) = (V^2/2)*(heaviside(w+1) - heaviside(w-1))    
J_elliptical(w,V) = 2*V^2*sqrt(Complex(1 -w^2))/pi;
heaviside(t) = 0.5 * (sign(t) + 1)

function reaction_mapping(w,J,L)
   
   # J = J*(heaviside(w+1) - heaviside(w-1)); # Ensure input function is clipped to the domain.
    
    #Define fixed numerical mesh over [-2,2] to capture spectral function and
    # its hilbert transform correctly within [-1,1].
    samp = 1000 # Number of points in mesh.
    x = LinRange(-2,2,samp);
    Jx = zeros(samp)
    Jx =[J(xi) for xi in x]  # Evaluate symbolic input function over the grid.

    tsq = zeros(1,L)
    en = zeros(1,L)
    # Loop over the omega intervals and perform integrations:
    Jcur = Jx; # Current bath spectral function.
    for s=1:L
      
      # Simple trapezoid integration for hopping squared and on-site energy:
      tsq[s] = trapz(x,Jcur); 
      en[s] = (1/tsq[s])*trapz(x,x.*Jcur);

      Jprev = Jcur;
      JH = imag(hilbert(Jprev)); # Hilbert transform.
      Jcur = ((tsq[s]/pi)^2)*Jprev./(JH.^2+Jprev.^2);
    end
    
    return tsq, en
end
    
    

function band_diag(B,d)
# Band-diagonalize matrix B with a bandwidth of d:
    n = size(B,1); # Assumed to be square.
    U = Diagonal(ones(n,n));
    for k=1:Int(floor(n/d)-1)
        C = B[(k*d+1):n,((k-1)*d+1):(k*d)]; # Extract coupling matrix.
        F = qr(C); # Upper-triangularize.
        blocks = [[Diagonal(ones(k*d,k*d))]; [F.Q']]
        Q = cat(blocks...,dims=(1,2))    # Form full triangularizing unitary.    
        B = Q*B*Q'; # Apply to input matrix to transform for next step.
        U = Q*U; # Save this step's unitary to the full sequence.
    end
    return B,U; # Return the final band-diagonalized matrix.
end;

function U_thermo(N,f_k)
    U_th = zeros(N,N)
    U_th[1,1],U_th[2,2] = 1,1
    b=0
    for i=3:2:N
        b += 1
        U_th[i,i],U_th[i+1,i+1] = sqrt(1-f_k[b]),-sqrt(1-f_k[b])
        U_th[i,i+1],U_th[i+1,i] = sqrt(f_k[b]),sqrt(f_k[b])
    end
    return U_th
end

function U_chain(N,U1,U2)

    U_tot = zeros(N,N)
    U_tot[1,1],U_tot[2,2] = 1,1
    b1 = 0
    for i=3:2:N
        b2 = 0 # resets the column iteration
        b1 +=1
        for j =3:2:N
            b2 += 1

            U_tot[i,j] = U1[b1,b2]
            U_tot[i+1,j+1] = U2[b1,b2]
        end
    end
    return U_tot
end


U_chain (generic function with 1 method)